In [1]:
from pylook.units import units
import pylook.calc as lc
from pylook.io import read_binary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

%matplotlib widget

In [2]:
data_path = 'p5517TPEclassl'

data, _ = read_binary(data_path)

data = pd.DataFrame.from_dict(data)

In [3]:
data['Time'] = np.cumsum(data['Time']/10000)

data['shr disp'] = data.pop('Vert DCDT')
data['shr stress'] = data.pop('Vert LOAD')

In [4]:
# SAMPLE TYPE: Westerly Granite, cylinder
# SAMPLE DIMENSIONS: Dia = 0.05225 m, height = 0.06 m
# SURFACE AREA = 
SA = (0.02525/2)**2 * np.pi


# CALIBRATIONS FOR DISPLACEMENTS AND STRESSES
#---------------------------------------------------------------------------------------

# VERTICAL DISPLACEMENT
# high gain 0.56957339 mm/V
v_disp_calib = (20/2**24)  * 0.56957339 * 1000 #um/mm 
data['shr disp'] = data['shr disp'] * v_disp_calib 

# VERTICAL LOAD
# low gain: 12.9 mV/kN
# area: 0.0022292545 m^2
v_load_calib = 12.9 * SA * (2**24/20)
data['shr stress'] = data['shr stress'] * 1/v_load_calib                                   

In [5]:
fig, ax1 = plt.subplots(1, 1, figsize=(10, 4)) # create figure
ax1.plot(data['shr disp'], '-r') # can replace with 'shr stress'
ax1.set_ylabel('shr disp ($\mu m$)')
ax1.set_xlabel('record #')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
data = data.drop(data.index[2546:3693]) # I don't need all of this data -- nothing is happening

In [7]:
# OFFSETS AND ZERO POINTS

# SHR DISP
data['shr disp'] = lc.zero(data['shr disp'], 668, mode='before')

# SHR STRESS
data['shr stress'] = lc.zero(data['shr stress'], 770, mode='before')

In [8]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True) # create figure with 2 subplots -- ax1, ax2
ax1.plot(data['Time'], data['shr disp'], color='darkolivegreen') # plot shr disp
ax2.plot(data['Time'], data['shr stress'], color='mediumblue') # plot shr stress
# AXES LABELS
ax1.set_ylabel('shear disp. ($\mu m$)')
ax2.set_xlabel('time (s)')
ax2.set_ylabel('shear stress (MPa)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# CALCULATE STRAIN
# e = (L0 - l)/L0
L0 = 60*1e3 #60 mm to um

data['strain'] = (1 - (L0 - data['shr disp'])/L0) #axial strain

In [10]:
fig, ax1 = plt.subplots(1, 1, figsize=(10, 8))
ax1.plot(data['strain'][0:2344]*100, data['shr stress'][0:2344], '-r') # multiple strain by 100 for unit percent
ax1.set_xlabel('axial strain (%)')
ax1.set_ylabel('axial stress (MPa)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# CALCULATE YOUNG'S MODULUS -- don't change

def YoungModCalc(dat_idx1, dat_idx2, idx1, idx2):

    str_idx1 = np.where(np.isclose(data['strain'][dat_idx1:dat_idx2], idx1, rtol=1e-2))[0][0] # get index for strain = idx1
    str_idx2 = np.where(np.isclose(data['strain'][dat_idx1:dat_idx2], idx2, rtol=1e-2))[0][0] # get index for strain = idx2

    p = np.polyfit(data['strain'][dat_idx1:dat_idx2][str_idx1:str_idx2], data['shr stress'][dat_idx1:dat_idx2][str_idx1:str_idx2], 1) # regression in the linear stress-strain region
    YoungMod = p[0]/1000 # this is the slope, Young's Modulus (GPa)

    ax_str = np.linspace(idx1, idx2, 100) # axial strain array
    YoungModFit = np.poly1d(p)(ax_str) # y-values from linear regression
    
    # PLOT THE RESULTS
    fig, ax1 = plt.subplots(1, 1, figsize=(10, 8))
    ax1.plot(data['strain'][dat_idx1:dat_idx2]*100, data['shr stress'][dat_idx1:dat_idx2], '-r') # multiple strain by 100 for unit percent
    ax1.plot(ax_str*100, YoungModFit, '-b', label='E = '+str(round(YoungMod,3))+' GPa')
    ax1.set_xlabel('axial strain (%)')
    ax1.set_ylabel('axial stress (MPa)')
    plt.legend()
    plt.show()
    
    return YoungMod, YoungModFit, ax_str

In [12]:
# indices 0 --> 2344 seems to be the most relevant window for the stress-strain curve

YoungMod, YoungModFit, ax_str = YoungModCalc(0, 2344, 0.008, 0.013)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …